In [1]:
import pandas as pd
import numpy as np
from autots import AutoTS
from sklearn.metrics import mean_absolute_error, mean_squared_error

# Load data
df = pd.read_csv("between_299_90_data.csv", parse_dates=["date"])

# Convert date column to datetime
df["date"] = pd.to_datetime(df["date"])

# Replace 'user_id_value' with the actual user ID you want to filter
user_id_value = "8b26cbbab26d33daba641fae9426d1bd25c1ad8e7abfe187a459512f5a8866e7"  # Example user ID

# Filter data for the specific user
user_data = df[df["profile_id"] == user_id_value]

# Extract year, month, and day
user_data["year"] = user_data["date"].dt.year
user_data["month"] = user_data["date"].dt.month
user_data["day"] = user_data["date"].dt.day

# Aggregate: Keep last follower count per day for each profile
daily_df = (
    user_data.sort_values(["profile_id", "date"])
    .groupby(["profile_id", "year", "month", "day"])
    .last()
    .reset_index()
)

# Convert back to date format
daily_df["date"] = pd.to_datetime(daily_df[["year", "month", "day"]])

# Define the full date range (daily from Jan 2023 to Dec 2024)
#full_date_range = pd.date_range(start="2023-01-01", end="2024-12-31", freq="D")  # 'D' for daily

# Drop unnecessary columns
daily_df = daily_df[["date", "profile_id", "followers_num"]]

# Create a MultiIndex with all profiles and all days
#full_index = pd.MultiIndex.from_product([unique_ids, full_date_range], names=["profile_id", "date"])

# Reindex the DataFrame to enforce this structure
#daily_df = daily_df.set_index(["profile_id", "date"]).reindex(full_index).reset_index()

# Randomly select 60 IDs
#selected_ids = np.random.choice(unique_ids, size=60, replace=False)

# Filter the DataFrame to include only the selected IDs
#daily_df = daily_df[daily_df['profile_id'].isin(selected_ids)]

# Verify the number of unique IDs in the filtered DataFrame
#print(f"Number of unique IDs in filtered DataFrame: {daily_df['profile_id'].nunique()}")

# Fill missing values
daily_df.fillna(method='ffill', inplace=True)
daily_df.fillna(method='bfill', inplace=True)
daily_df.fillna(0, inplace=True)

# # Split the data
# train_df = daily_df[daily_df['date'] < '2024-07-01']
# test_df = daily_df[daily_df['date'] >= '2024-07-01']

# Sort data by date to ensure correct splitting
daily_df = daily_df.sort_values("date")

# Find the total number of unique days
unique_dates = daily_df["date"].unique()

# Determine the split index (75% of the data)
split_index = int(len(unique_dates) * 0.75)

# Get the corresponding date for the split
split_date = unique_dates[split_index]  # This is the cutoff date

# Perform the split dynamically
train_df = daily_df[daily_df["date"] < split_date]
test_df = daily_df[daily_df["date"] >= split_date]

# Verify the split
print("Train Data Range:", train_df["date"].min(), "to", train_df["date"].max())
print("Test Data Range:", test_df["date"].min(), "to", test_df["date"].max())
print(f"Split Date: {split_date}, Total Unique Days: {len(unique_dates)}")


# Pivot and convert dates to daily format
def prepare_for_autts(df):
    df = df.pivot(index='date', columns='profile_id', values='followers_num')
    df = df.fillna(method='ffill')
    df.index = df.index.to_period('D').to_timestamp('D')  # Convert to daily timestamp
    df = df.fillna(method='ffill')
    return df

train_df = prepare_for_autts(train_df)
test_df = prepare_for_autts(test_df)

# Ensure no missing values
train_df.fillna(method='ffill', inplace=True)
train_df.fillna(method='bfill', inplace=True)
train_df.fillna(0, inplace=True)

test_df.fillna(method='ffill', inplace=True)
test_df.fillna(method='bfill', inplace=True)
test_df.fillna(0, inplace=True)

# ----- AUTOTS MODEL -----
# Initialize and train the AutoTS model for daily forecasting
model = AutoTS(
    forecast_length=len(test_df),  
    frequency='D',  # Daily frequency
    ensemble='simple',
    max_generations=10,
    num_validations=3,
    validation_method='backwards'
)

# Train the model
model = model.fit(train_df)

# Generate predictions
predictions = model.predict(prediction_interval=0.95, fail_on_forecast_nan=False)
forecast_df = predictions.forecast.fillna(0)

common_index = forecast_df.index.intersection(test_df.index)
forecast_df = forecast_df.loc[common_index]
test_df_aligned = test_df.loc[common_index]

# Align test data with predictions
test_df_aligned = test_df.loc[forecast_df.index]

# Evaluate AutoTS model
evaluation_results_90 = []
for profile_id in test_df_aligned.columns:
    true_values = test_df_aligned[profile_id]
    predicted_values = forecast_df[profile_id]
    mae_autots = mean_absolute_error(true_values, predicted_values)
    mse_autots = mean_squared_error(true_values, predicted_values)
    evaluation_results_90.append({'Profile': profile_id, 'MAE_AutoTS_Daily': mae_autots, 'MSE_AutoTS_Daily': mse_autots})

# Create a comparison DataFrame
evaluation_df_90 = pd.DataFrame(evaluation_results_90)

Train Data Range: 2023-01-01 00:00:00 to 2023-08-04 00:00:00
Test Data Range: 2023-08-05 00:00:00 to 2024-10-15 00:00:00
Split Date: 2023-08-05 00:00:00, Total Unique Days: 288
Using 4 cpus for n_jobs.
Data frequency is: D, used frequency is: D
Too many training validations for length of data provided, decreasing num_validations to 1
Model Number: 1 with model AverageValueNaive in generation 0 of 10
Model Number: 2 with model AverageValueNaive in generation 0 of 10
Model Number: 3 with model AverageValueNaive in generation 0 of 10


/tmp/ipykernel_81335/3274799494.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  user_data["year"] = user_data["date"].dt.year
/tmp/ipykernel_81335/3274799494.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  user_data["month"] = user_data["date"].dt.month
/tmp/ipykernel_81335/3274799494.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.

Model Number: 4 with model DatepartRegression in generation 0 of 10
Model Number: 5 with model DatepartRegression in generation 0 of 10
Model Number: 6 with model DatepartRegression in generation 0 of 10


/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/sklearn/svm/_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/sklearn/svm/_base.py:1250: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:546: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


Model Number: 7 with model DatepartRegression in generation 0 of 10
Template Eval Error: ImportError('Tensorflow not available, install with pip install tensorflow.') in model 7 in generation 0: DatepartRegression
Model Number: 8 with model ETS in generation 0 of 10
Model Number: 9 with model ETS in generation 0 of 10
Model Number: 10 with model GLM in generation 0 of 10
Model Number: 11 with model GLM in generation 0 of 10
Model Number: 12 with model GLS in generation 0 of 10
Model Number: 13 with model GLS in generation 0 of 10
Model Number: 14 with model LastValueNaive in generation 0 of 10
Model Number: 15 with model LastValueNaive in generation 0 of 10
Model Number: 16 with model LastValueNaive in generation 0 of 10
Model Number: 17 with model LastValueNaive in generation 0 of 10
Model Number: 18 with model SeasonalNaive in generation 0 of 10
Model Number: 19 with model SeasonalNaive in generation 0 of 10
Model Number: 20 with model SeasonalNaive in generation 0 of 10
Model Number

...

71 - Ensemble with avg smape 10.37: 
72 - Ensemble with avg smape 10.37: 
73 - Ensemble with avg smape 10.43: 
74 - Ensemble with avg smape 10.48: 
75 - Ensemble with avg smape 11.19: 
76 - Ensemble with avg smape 11.34: 
77 - Ensemble with avg smape 11.07: 
78 - Ensemble with avg smape 11.07: 
79 - Ensemble with avg smape 10.37: 
80 - Ensemble with avg smape 11.14: 
81 - Ensemble with avg smape 10.39: 
82 - Ensemble with avg smape 10.37: 
83 - Ensemble with avg smape 18.24: 
84 - Ensemble with avg smape 10.37: 
Model Number: 85 of 8 with model Ensemble for Validation 1


/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/statsmodels/genmod/families/family.py:1367: ValueWarning: Negative binomial dispersion parameter alpha not set. Using default value alpha=1.0.
  warnings.warn("Negative binomial dispersion parameter alpha not "
/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/statsmodels/genmod/families/family.py:1402: RuntimeWarning: divide by zero encountered in divide
  endog_mu = self._clean(endog / mu)
/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/autots/tools/seasonal.py:896: RuntimeWarning: invalid value encountered in divide
  scores = np.mean(np.abs(a - b) / divisor, axis=2)
/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/autots/tools/seasonal.py:896: RuntimeWarning: invalid value encountered in divide
  scores = np.mean(np.abs(a - b) / divisor, axis=2)
/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/autots/tools/seasonal.py:896: RuntimeWarning: invali

85 - Ensemble with avg smape 10.37: 
86 - Ensemble with avg smape 10.37: 
87 - Ensemble with avg smape 10.44: 
88 - Ensemble with avg smape 10.48: 
89 - Ensemble with avg smape 11.19: 
90 - Ensemble with avg smape 11.34: 
91 - Ensemble with avg smape 11.07: 
92 - Ensemble with avg smape 11.07: 
93 - Ensemble with avg smape 10.37: 
94 - Ensemble with avg smape 11.14: 
95 - Ensemble with avg smape 10.4: 
96 - Ensemble with avg smape 10.37: 
97 - Ensemble with avg smape 18.23: 
98 - Ensemble with avg smape 10.37: 
Model Number: 99 of 8 with model Ensemble for Validation 1


/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/statsmodels/genmod/families/family.py:1367: ValueWarning: Negative binomial dispersion parameter alpha not set. Using default value alpha=1.0.
  warnings.warn("Negative binomial dispersion parameter alpha not "
/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/statsmodels/genmod/families/family.py:1402: RuntimeWarning: divide by zero encountered in divide
  endog_mu = self._clean(endog / mu)
/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/autots/tools/seasonal.py:896: RuntimeWarning: invalid value encountered in divide
  scores = np.mean(np.abs(a - b) / divisor, axis=2)
/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/autots/tools/seasonal.py:896: RuntimeWarning: invalid value encountered in divide
  scores = np.mean(np.abs(a - b) / divisor, axis=2)
/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/autots/tools/seasonal.py:896: RuntimeWarning: invali

99 - Ensemble with avg smape 10.37: 
100 - Ensemble with avg smape 10.37: 
101 - Ensemble with avg smape 10.45: 
102 - Ensemble with avg smape 10.48: 
103 - Ensemble with avg smape 11.19: 
104 - Ensemble with avg smape 11.34: 
105 - Ensemble with avg smape 11.07: 
106 - Ensemble with avg smape 11.07: 
107 - Ensemble with avg smape 10.37: 
108 - Ensemble with avg smape 11.14: 
109 - Ensemble with avg smape 10.4: 
110 - Ensemble with avg smape 10.37: 
111 - Ensemble with avg smape 18.23: 
112 - Ensemble with avg smape 10.37: 


In [2]:
evaluation_results_90

[{'Profile': '8b26cbbab26d33daba641fae9426d1bd25c1ad8e7abfe187a459512f5a8866e7',
  'MAE_AutoTS_Daily': 1878505.0863176002,
  'MSE_AutoTS_Daily': 3940686154692.1245}]

In [3]:
import pandas as pd
import numpy as np
from autots import AutoTS
from sklearn.metrics import mean_absolute_error, mean_squared_error

# Load data
df = pd.read_csv("between_299_90_data.csv", parse_dates=["date"])

# Convert date column to datetime
df["date"] = pd.to_datetime(df["date"])

# Replace 'user_id_value' with the actual user ID you want to filter
user_id_value = "8b26cbbab26d33daba641fae9426d1bd25c1ad8e7abfe187a459512f5a8866e7"  # Example user ID

# Filter data for the specific user
user_data = df[df["profile_id"] == user_id_value]

# Extract year, month, and day
user_data["year"] = user_data["date"].dt.year
user_data["month"] = user_data["date"].dt.month
user_data["day"] = user_data["date"].dt.day

# Aggregate: Keep last follower count per day for each profile
daily_df = (
    user_data.sort_values(["profile_id", "date"])
    .groupby(["profile_id", "year", "month", "day"])
    .last()
    .reset_index()
)

# Convert back to date format
daily_df["date"] = pd.to_datetime(daily_df[["year", "month", "day"]])

# Define the full date range (daily from Jan 2023 to Dec 2024)
#full_date_range = pd.date_range(start="2023-01-01", end="2024-12-31", freq="D")  # 'D' for daily

# Drop unnecessary columns
daily_df = daily_df[["date", "profile_id", "followers_num"]]

# Create a MultiIndex with all profiles and all days
#full_index = pd.MultiIndex.from_product([unique_ids, full_date_range], names=["profile_id", "date"])

# Reindex the DataFrame to enforce this structure
#daily_df = daily_df.set_index(["profile_id", "date"]).reindex(full_index).reset_index()

# Randomly select 60 IDs
#selected_ids = np.random.choice(unique_ids, size=60, replace=False)

# Filter the DataFrame to include only the selected IDs
#daily_df = daily_df[daily_df['profile_id'].isin(selected_ids)]

# Verify the number of unique IDs in the filtered DataFrame
#print(f"Number of unique IDs in filtered DataFrame: {daily_df['profile_id'].nunique()}")

# Fill missing values
daily_df.fillna(method='ffill', inplace=True)
daily_df.fillna(method='bfill', inplace=True)
daily_df.fillna(0, inplace=True)

# # Split the data
# train_df = daily_df[daily_df['date'] < '2024-07-01']
# test_df = daily_df[daily_df['date'] >= '2024-07-01']

# Sort data by date to ensure correct splitting
daily_df = daily_df.sort_values("date")

# Find the total number of unique days
unique_dates = daily_df["date"].unique()

# Determine the split index (75% of the data)
split_index = int(len(unique_dates) * 0.75)

# Get the corresponding date for the split
split_date = unique_dates[split_index]  # This is the cutoff date

# Perform the split dynamically
train_df = daily_df[daily_df["date"] < split_date]
test_df = daily_df[daily_df["date"] >= split_date]

# Verify the split
print("Train Data Range:", train_df["date"].min(), "to", train_df["date"].max())
print("Test Data Range:", test_df["date"].min(), "to", test_df["date"].max())
print(f"Split Date: {split_date}, Total Unique Days: {len(unique_dates)}")


# Pivot and convert dates to daily format
def prepare_for_autts(df):
    df = df.pivot(index='date', columns='profile_id', values='followers_num')
    df = df.fillna(method='ffill')
    df.index = df.index.to_period('D').to_timestamp('D')  # Convert to daily timestamp
    df = df.fillna(method='ffill')
    return df

train_df = prepare_for_autts(train_df)
test_df = prepare_for_autts(test_df)

# Ensure no missing values
train_df.fillna(method='ffill', inplace=True)
train_df.fillna(method='bfill', inplace=True)
train_df.fillna(0, inplace=True)

test_df.fillna(method='ffill', inplace=True)
test_df.fillna(method='bfill', inplace=True)
test_df.fillna(0, inplace=True)

# ----- AUTOTS MODEL -----
# Initialize and train the AutoTS model for daily forecasting
model = AutoTS(
    forecast_length=len(test_df),  
    frequency='D',  # Daily frequency
    ensemble='simple',
    max_generations=10,
    num_validations=3,
    validation_method='backwards'
)

# Train the model
model = model.fit(train_df)

# Generate predictions
predictions = model.predict(prediction_interval=0.95, fail_on_forecast_nan=False)
forecast_df = predictions.forecast.fillna(0)

common_index = forecast_df.index.intersection(test_df.index)
forecast_df = forecast_df.loc[common_index]
test_df_aligned = test_df.loc[common_index]

# Align test data with predictions
test_df_aligned = test_df.loc[forecast_df.index]

# Evaluate AutoTS model
evaluation_results_90 = []
for profile_id in test_df_aligned.columns:
    true_values = test_df_aligned[profile_id]
    predicted_values = forecast_df[profile_id]
    mae_autots = mean_absolute_error(true_values, predicted_values)
    mse_autots = mean_squared_error(true_values, predicted_values)
    evaluation_results_90.append({'Profile': profile_id, 'MAE_AutoTS_Daily': mae_autots, 'MSE_AutoTS_Daily': mse_autots})

# Create a comparison DataFrame
evaluation_df_90 = pd.DataFrame(evaluation_results_90)

evaluation_results_90

Train Data Range: 2023-01-01 00:00:00 to 2023-08-04 00:00:00
Test Data Range: 2023-08-05 00:00:00 to 2024-10-15 00:00:00
Split Date: 2023-08-05 00:00:00, Total Unique Days: 288
Using 4 cpus for n_jobs.
Data frequency is: D, used frequency is: D
Too many training validations for length of data provided, decreasing num_validations to 1
Model Number: 1 with model AverageValueNaive in generation 0 of 10
Model Number: 2 with model AverageValueNaive in generation 0 of 10
Model Number: 3 with model AverageValueNaive in generation 0 of 10
Model Number: 4 with model DatepartRegression in generation 0 of 10


/tmp/ipykernel_81335/2315399121.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  user_data["year"] = user_data["date"].dt.year
/tmp/ipykernel_81335/2315399121.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  user_data["month"] = user_data["date"].dt.month
/tmp/ipykernel_81335/2315399121.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.

Model Number: 5 with model DatepartRegression in generation 0 of 10
Model Number: 6 with model DatepartRegression in generation 0 of 10


/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/sklearn/svm/_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/sklearn/svm/_base.py:1250: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


Model Number: 7 with model DatepartRegression in generation 0 of 10
Template Eval Error: ImportError('Tensorflow not available, install with pip install tensorflow.') in model 7 in generation 0: DatepartRegression
Model Number: 8 with model ETS in generation 0 of 10


...

/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/statsmodels/genmod/families/links.py:198: RuntimeWarning: overflow encountered in exp
  t = np.exp(-z)


Model Number: 1561 with model Ensemble in generation 12 of Ensembles
Model Number: 1575 with model Ensemble in generation 12 of Ensembles
Validation Round: 1
Model Number: 1 of 8 with model Ensemble for Validation 1
📈 1 - Ensemble with avg smape 10.43: 
2 - Ensemble with avg smape 10.43: 
📈 3 - Ensemble with avg smape 10.39: 
4 - Ensemble with avg smape 10.43: 
5 - Ensemble with avg smape 11.19: 
6 - Ensemble with avg smape 11.34: 
7 - Ensemble with avg smape 11.07: 
8 - Ensemble with avg smape 11.07: 
9 - Ensemble with avg smape 10.43: 
10 - Ensemble with avg smape 11.14: 
📈 11 - Ensemble with avg smape 10.38: 
12 - Ensemble with avg smape 10.41: 
13 - Ensemble with avg smape 18.33: 
14 - Ensemble with avg smape 10.4: 
Model Number: 15 of 8 with model Ensemble for Validation 1
15 - Ensemble with avg smape 10.42: 
16 - Ensemble with avg smape 10.42: 
17 - Ensemble with avg smape 10.38: 
18 - Ensemble with avg smape 10.43: 
19 - Ensemble with avg smape 11.19: 
20 - Ensemble with avg sma

/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/statsmodels/genmod/generalized_linear_model.py:1342: PerfectSeparationWarning: Perfect separation or prediction detected, parameter may not be identified
  warnings.warn(msg, category=PerfectSeparationWarning)
/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/statsmodels/genmod/generalized_linear_model.py:1342: PerfectSeparationWarning: Perfect separation or prediction detected, parameter may not be identified
  warnings.warn(msg, category=PerfectSeparationWarning)
/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/statsmodels/genmod/generalized_linear_model.py:1342: PerfectSeparationWarning: Perfect separation or prediction detected, parameter may not be identified
  warnings.warn(msg, category=PerfectSeparationWarning)
/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/statsmodels/genmod/generalized_linear_model.py:1342: PerfectSeparationWarning: Perfect separation or p

71 - Ensemble with avg smape 10.41: 
72 - Ensemble with avg smape 10.41: 
73 - Ensemble with avg smape 10.38: 
74 - Ensemble with avg smape 10.42: 
75 - Ensemble with avg smape 11.19: 
76 - Ensemble with avg smape 11.34: 
77 - Ensemble with avg smape 11.07: 
78 - Ensemble with avg smape 11.07: 
79 - Ensemble with avg smape 10.41: 
80 - Ensemble with avg smape 11.14: 
81 - Ensemble with avg smape 10.38: 
82 - Ensemble with avg smape 10.4: 
83 - Ensemble with avg smape 18.32: 
84 - Ensemble with avg smape 10.4: 
Model Number: 85 of 8 with model Ensemble for Validation 1
85 - Ensemble with avg smape 10.46: 
86 - Ensemble with avg smape 10.46: 
87 - Ensemble with avg smape 10.39: 
88 - Ensemble with avg smape 10.46: 
89 - Ensemble with avg smape 11.19: 
90 - Ensemble with avg smape 11.34: 
91 - Ensemble with avg smape 11.07: 
92 - Ensemble with avg smape 11.07: 
93 - Ensemble with avg smape 10.46: 
94 - Ensemble with avg smape 11.14: 
95 - Ensemble with avg smape 10.4: 
96 - Ensemble with 

[{'Profile': '8b26cbbab26d33daba641fae9426d1bd25c1ad8e7abfe187a459512f5a8866e7',
  'MAE_AutoTS_Daily': 348091.3009912918,
  'MSE_AutoTS_Daily': 314505231559.30615}]